In [2]:
import psycopg2

# Revision de la tabla de tags y de posts

In [12]:
connection = psycopg2.connect(
    user="postgres",
    password="postgres",
    host="127.0.0.1",
    port="63333",
    database="stack_overflow",
)

In [4]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select tags from posts_19 where tags is not null limit 6")
        posts_tags_sample = cursor.fetchall()
posts_tags_sample

[('<bash><whitespace>',),
 ('<visual-studio-2008><validation><css>',),
 ('<javascript><apache-flex><actionscript-3>',),
 ('<sql-server><indexing>',),
 ('<c++><boost>',),
 ('<performance><http><https>',)]

In [5]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute("select tag_name, count from tags order by count desc limit 6")
        top_tags_sample = cursor.fetchall()
top_tags_sample

[('javascript', 2453736),
 ('python', 2071403),
 ('java', 1878099),
 ('c#', 1571218),
 ('php', 1451338),
 ('android', 1393144)]

In [6]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            "select tag_name from tags where tag_name like '%<%' or tag_name like '%>%'"
        )
        tags_with_angle_brackets = cursor.fetchall()
tags_with_angle_brackets

[]

# Crear la tabla de posts_tags

In [7]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
        with posts_tags_exp as (
            select id, regexp_split_to_table(trim('<>' from tags), '><') as i_tag
            from posts_19 where tags is not null limit 10
        )
        select pt.id, pt.i_tag, t.id
        from posts_tags_exp pt
        left join tags t on pt.i_tag=t.tag_name
        """
        )
        posts2tags = cursor.fetchall()
posts2tags

[(149206, 'javascript', 3),
 (149131, 'css', 4),
 (149206, 'apache-flex', 38),
 (149213, 'sql-server', 72),
 (149213, 'indexing', 317),
 (149131, 'validation', 355),
 (149057, 'bash', 387),
 (149057, 'whitespace', 1084),
 (149206, 'actionscript-3', 14012),
 (149131, 'visual-studio-2008', 32472)]

In [14]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """
        DROP TABLE IF EXISTS posts_19_tags;
        CREATE TABLE posts_19_tags AS
        with posts_tags_exp as (
            select id, regexp_split_to_table(trim('<>' from tags), '><') as i_tag
            from posts_19 where tags is not null
        )
        select pt.id as post_id, t.id as tag_id
        from posts_tags_exp pt
        left join tags t on pt.i_tag=t.tag_name
        """
        )

### Prueba de la relacion

In [16]:
with connection:
    with connection.cursor() as cursor:
        cursor.execute(
            """select t.tag_name, p.id, u.country
        from posts_19_tags pt
        join posts_19 p on pt.post_id=p.id 
        join tags t on pt.tag_id=t.id 
        join users u on p.owner_user_id=u.id
        where u.country is not null
        limit 20"""
        )
        post_tags_country = cursor.fetchall()
post_tags_country

[('maven-surefire-plugin', 74306381, 'Ukraine'),
 ('lombok', 74306381, 'Ukraine'),
 ('reporting', 74306381, 'Ukraine'),
 ('maven', 74306381, 'Ukraine'),
 ('delimiter', 74308951, 'Cameroon'),
 ('split', 74308951, 'Cameroon'),
 ('string', 74308951, 'Cameroon'),
 ('python', 74308951, 'Cameroon'),
 ('stack-navigator', 74309130, 'Bulgaria'),
 ('react-native', 74309130, 'Bulgaria'),
 ('reactjs', 74309130, 'Bulgaria'),
 ('android', 74309130, 'Bulgaria'),
 ('javascript', 74309130, 'Bulgaria'),
 ('husky', 74309392, 'United States'),
 ('githooks', 74309392, 'United States'),
 ('visual-studio-code', 74309392, 'United States'),
 ('git', 74309392, 'United States'),
 ('reactjs', 74309392, 'United States'),
 ('ssh-keys', 74316439, 'Latvia'),
 ('key', 74316439, 'Latvia')]